# Eye Detection
using opencv-python, take a photo of a person from their webcam and create a circle around their eyes.

citations:
<br>
[0]: https://www.codegrepper.com/code-examples/python/how+to+open+webcam+with+python
    how to open webcam with python, (note the check to make sure it isn't already open.)
<br>
[1]: https://stackoverflow.com/questions/35003476/opencv-python-how-to-detect-if-a-window-is-closed/37881722#37881722
    how to close cv2 window with the x button
<br>
[2]: https://www.programiz.com/python-programming/datetime
    formatting date and time
<br>
[3]: https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Basics.html
    using widgets in python notebook
<br>
[4]: https://docs.opencv.org/4.x/dd/d1a/group__imgproc__feature.html#ga47849c3be0d0406ad3ca45db65a25d2d
    using hough circles

In [1]:
import cv2
import numpy as np
from datetime import datetime
import ipywidgets as widgets
from IPython.display import display
import glob
import matplotlib.pyplot as plt


In [2]:
# for taking photos to capture the face
cap = cv2.VideoCapture(0) # opening webcam [0]

# Check if the webcam is opened correctly
if not cap.isOpened():
    raise IOError("Cannot open webcam")

try:
    cv2.namedWindow("Input", cv2.WINDOW_AUTOSIZE)
    while cv2.getWindowProperty('Input', 0) >= 0: # closing with x [1]
        ret, frame = cap.read()
        # frame = cv2.resize(frame, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_AREA)
        cv2.imshow('Input', frame)

        c = cv2.waitKey(1)
        if c == ord('s'):
            timestamp = datetime.now() # format datetime [2]
            time = timestamp.strftime("%Y%m%d_%H%M%S")
            cv2.imwrite(f"{time}_face.png", frame)
        if c == ord('q'):
            break
finally:
    cap.release()
    cv2.destroyAllWindows()

KeyboardInterrupt: 

In [40]:
# for drawing edges on the face with canny edge detector
def drive_canny(image_url, canny_high_thresh):
    img = cv2.imread(image_url)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, canny_high_thresh / 2, canny_high_thresh)
    fig = plt.figure()
    fig.set_size_inches(12, 12)
    plt.imshow(edges, cmap='gray')
    plt.show()
    # change figure size
    return edges

image_urls = glob.glob("*.png")
if len(image_urls) == 0:
    raise ValueError("No image files found")
display(
    widgets.interactive(
        drive_canny,
        image_url=widgets.Dropdown(
            options=image_urls,
            value=image_urls[0],
        ),
        canny_high_thresh=widgets.IntSlider(
            value=150,
            min=0,
            max=255,
            step=1,
        ),
    )
)


interactive(children=(Dropdown(description='image_url', options=('20220609_202103_face.png', '20220609_202111_…

In [49]:
# detecting circles in edge images with cv2
def drive_circles(image_url, method, accumulator_ratio, minDist, param1, param2, min_radius, max_radius): # using cv2.HoughCircles [4]
    img = cv2.imread(image_url)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (7, 7), 1.5)
    edges = cv2.Canny(blur, param1 / 2, param1)
    circles = cv2.HoughCircles(blur, method, accumulator_ratio, minDist, param1=param1, param2=param2, minRadius=min_radius, maxRadius=max_radius)
    if circles is not None:
        circles = np.uint16(np.around(circles))
        for i in circles[0, :]:
            cv2.circle(img, (i[0], i[1]), i[2], (0, 255, 0), 2)
            cv2.circle(img, (i[0], i[1]), 2, (0, 0, 255), 3)
    
    fig, axs = plt.subplots(3, 1, figsize=(32, 12))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    axs[0].imshow(blur, cmap='gray')
    axs[0].set_title('Blurred')
    axs[1].imshow(edges)
    axs[1].set_title('edges')
    axs[2].imshow(img)
    axs[2].set_title(f'Detected Circles\nnum:{len(circles[0]) if circles is not None else 0}')
    fig.tight_layout()
    plt.show()
    # change figure size
    return circles

# creating an interactive widget for changing the parameters of the hough circle detection [3]
display(
    widgets.interactive(
        drive_circles,
        image_url=widgets.Dropdown(
            options=image_urls,
            value=image_urls[0],
        ),
        method=widgets.Dropdown(
            options=[cv2.HOUGH_GRADIENT, cv2.HOUGH_GRADIENT_ALT],
            value=cv2.HOUGH_GRADIENT_ALT,
        ),
        accumulator_ratio=widgets.FloatSlider(
            value=.8,
            min=0,
            max=3,
            step=0.1,
        ),
        minDist=widgets.FloatSlider(
            value=20,
            min=0,
            max=1000, # should be the distance between the eyes in pixels
            step=1,
        ),
        param1=widgets.FloatSlider(
            value=42,
            min=0,
            max=255,
        ),
        param2=widgets.FloatSlider(
            value=.75,
            min=0,
            max=30,
        ),
        min_radius=widgets.IntSlider(
            value=5,
            min=0,
            max=100,
            step=1,
        ),
        max_radius=widgets.IntSlider(
            value=15,
            min=0,
            max=100,
            step=1,
        ),
    )
)
# with method 3, ie cv2.HOUGH_GRADIENT,
# accumulator .1, minDist 20, param1 86, param2 13, min_radius 3, max_radius 15
# detected too many circles, ~ 15 per image, but always got eyes

# with method 4, ie cv2.HOUGH_GRADIENT_ALT,
# accumulator_ratio .8, minDist 20, param1 42, param2 .75, min_radius 5, max_radius 15
# detected too many circles, ~ 3 per image, but always got eyes, so much better!

interactive(children=(Dropdown(description='image_url', options=('20220609_202103_face.png', '20220609_202111_…

In [51]:
# for running eye detection real time
cap = cv2.VideoCapture(0) # opening webcam [0]

# Check if the webcam is opened correctly
if not cap.isOpened():
    raise IOError("Cannot open webcam")

try:
    cv2.namedWindow("Input", cv2.WINDOW_AUTOSIZE)
    while cv2.getWindowProperty('Input', 0) >= 0: # closing with x [1]
        ret, frame = cap.read()

        img = frame
        method = cv2.HOUGH_GRADIENT_ALT
        accumulator_ratio = .8
        minDist =  20
        param1 =  42
        param2 =  .75
        min_radius = 5
        max_radius = 15

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(gray, (7, 7), 1.5)
        edges = cv2.Canny(blur, param1 / 2, param1)
        circles = cv2.HoughCircles(blur, method, accumulator_ratio, minDist, param1=param1, param2=param2, minRadius=min_radius, maxRadius=max_radius)
        if circles is not None:
            circles = np.uint16(np.around(circles))
            for i in circles[0, :]:
                cv2.circle(img, (i[0], i[1]), i[2], (0, 255, 0), 2)
                cv2.circle(img, (i[0], i[1]), 2, (0, 0, 255), 3)
        
        cv2.imshow('Input', frame)

        c = cv2.waitKey(1)
        if c == ord('s'):
            timestamp = datetime.now() # format datetime [2]
            time = timestamp.strftime("%Y%m%d_%H%M%S")
            cv2.imwrite(f"{time}_face_circled.png", frame)
        if c == ord('q'):
            break
finally:
    cap.release()
    cv2.destroyAllWindows()